# 独热编码 (One-Hot Encoding)
独热编码 (One-Hot Encoding) 看起来是最基础的数据预处理手段，但我们不能只把它看作简单的 get_dummies。

我们要从 线性代数空间 和 计算图 (Computational Graph) 的角度去理解它：它是连接“离散符号世界”与“连续向量世界”的桥梁，也是所有 Embedding 技术的数学源头。

## 1 概念与数学直觉

**为什么不能直接用数字？**
假设我们有三种类别： [猫, 狗, 鸟]。

如果使用 Label Encoding 编码为 [1, 2, 3]：模型会认为

1. $3 > 2 > 1$（鸟比狗“大”？）
2. $\text{Distance}(猫, 鸟) = 2$，而 $\text{Distance}(猫, 狗) = 1$

独热编码的几何意义我们将每个类别映射到一个 $N$ 维欧几里得空间的基向量 (Basis Vector) 上。
若类别数为 $K$，对于第 $i$ 个类别，其向量 $e_i$ 为：
$$e_i = [0, \dots, 1, \dots, 0] \in \mathbb{R}^K$$

其中仅第 $i$ 位为 1

核心性质：正交性 (Orthogonality) & 稀疏性 (Sparsity)
这是独热编码最重要的数学性质：
$$e_i^T \cdot e_j = 0 \quad (\text{if } i \neq j)$$
$$||e_i - e_j||_2 = \sqrt{2} \quad (\forall i \neq j)$$

- 在独热空间中，所有类别之间的距离都是相等的（$\sqrt{2}$），且彼此正交（无关）。这消除了序数偏见。
- 语义鸿沟。因为点积为 0，模型无法通过 $x_i \cdot x_j$ 学到“猫”和“狗”比“猫”和“桌子”更相似。
- 稀疏矩阵-种类多，则矩阵为 $N*N$

## 2 深入实践

### 2.1 Phase 1: Numpy 手写底层（理解内存与索引）

In [1]:
import numpy as np
class OneHotEncoder:
    def __init__(self, num_classes):
        self.num_classes = num_classes
    def forward(self, indices):
        batch_size = indices.shape[0]
        # 创建 batch_size x num_classes 的零矩阵
        one_hot = np.zeros((batch_size, self.num_classes), dtype=np.float32)
        # [0,1],[1,4],[2,0] 位置赋值为1（花式索引：按轴分别指定索引）
        one_hot[np.arange(batch_size), indices] = 1.0
        return one_hot

    def backward(self, grad_output):
        return None

# 结果分类个数
encoder = OneHotEncoder(num_classes=5)
# 输入样本类别，0号样本是类1，1号样本是类4，2号样本是类0
indices = np.array([1,4,0], dtype=np.int32)
output = encoder.forward(indices)

print(f"indices: {indices}")
print(f"one-hot output:\n{output}")
print(f"dot product of sample 0 and 1: {np.dot(output[0], output[1])}")

indices: [1 4 0]
one-hot output:
[[0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]]
dot product of sample 0 and 1: 0.0


### 2.2 Phase 2: PyTorch 中One-Hot

In [2]:
import torch
import torch.nn.functional as F

def industrial_one_hot(indices, num_classes, lable_smoothing=0.0):
    # PyTorch 的 one_hot 函数
    one_hot = F.one_hot(indices, num_classes=num_classes).float()
    if lable_smoothing > 0:
        one_hot = one_hot * (1 - lable_smoothing) + lable_smoothing / num_classes
    return one_hot

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
y_true = torch.tensor([0, 2], device=device)
num_classes = 4
y_encoded = industrial_one_hot(y_true, num_classes, lable_smoothing=0.1)
print(f"Device: {y_encoded.device}, One-hot with label smoothing:\n{y_encoded}")

Device: cpu, One-hot with label smoothing:
tensor([[0.9250, 0.0250, 0.0250, 0.0250],
        [0.0250, 0.0250, 0.9250, 0.0250]])


### 2.3 Phase 3: 独热编码的陷阱
1. 维度问题
- 如果词表有5w个词，One-Hot向量长度就是5w
- 一个Batch 64个句子，每个句子100个词。Tensor 大小: $64 \times 100 \times 50000 \times 4 \text{bytes} \approx 1.28 \text{GB}$
显存瞬间打爆，改用 nn.Embedding （查表法）
2. 类索引越界
- 如果num_classes=10,但输入错误比如 10或-1
- F.one_hot会报错或扩充为度，导致后面的linear层形状不匹配崩溃
输入前 assert indices.max() < num_classes

## 3 落地架构
真实的推荐系统或者LLM中，One-Hot仅仅是逻辑概念，物理实现通常被优化掉。
核心：One-Hot + Linear = Embedding

这是你需要刻在脑子里的公式。假设 $x$ 是 shape 为 $(1, V)$ 的 One-Hot 向量，$W$ 是 shape 为 $(V, D)$ 的权重矩阵。

$$x \cdot W = [0, \dots, 1_i, \dots, 0] \cdot \begin{bmatrix} w_{0} \\ \vdots \\ w_{i} \\ \vdots \\ w_{V-1} \end{bmatrix} = w_{i}$$

矩阵乘法等价于查表(Look-up):
1. 类别少：星期几/性别 -> 直接用 One-Hot 
2. 类别多：UserID/WordID -> nn.Embedding(PyTorch) 或 tf.gather(TF)，底层只进行内存寻址拷贝，不进行矩阵乘法

### 3.1 nn.Embedding 解决 One-Hot 问题
首先一次内存寻址，复杂度为 $O(1)$ ，矩阵乘法需要 $V*D$ 次浮点运算，浪费算力去乘一堆0。
内存上，将稀疏向量压缩为稠密向量，只存储有效信息。
语义空间更合理，One-Hot任意两个词垂直，Embedding是可学习参数，训练后“猫”和“狗”向量在空间中会靠的很近，因为他们在类似的上下文中出现。
连续可导，Embedding可以计算梯度并更新。

下面手写一个支持反向传播的 Embedding 层：

In [4]:
import numpy as np
class MyEmbedding:
    def __init__(self, num_embeddings, embedding_dim):
        self.num_embeddings = num_embeddings
        self.embedding_dim = embedding_dim
        # 随机初始化嵌入矩阵
        self.weight = np.random.randn(num_embeddings, embedding_dim).astype(np.float32)
        self.last_indices = None

    def forward(self, indices):
        self.last_indices = indices
        # 使用花式索引获取嵌入向量
        embedded = self.weight[indices]
        return embedded
    
    def backward(self, grad_output):
        grad_weight = np.zeros_like(self.weight)
        flat_indices = self.last_indices.reshape(-1)
        flat_grad = grad_output.reshape(-1, self.embedding_dim)
        np.add.at(grad_weight, flat_indices, flat_grad)
        return grad_weight

# 创建嵌入层，词汇表大小10，嵌入维度3
embedding = MyEmbedding(num_embeddings=5, embedding_dim=3)
# 输入样本
input_indices = np.array([[1, 2], [1, 4]])
output = embedding.forward(input_indices)
print("forward output:\n", output)

fake_grad = np.ones_like(output)
dw = embedding.backward(fake_grad)
print("gradient w.r.t. embedding weights:\n", dw)


forward output:
 [[[ 0.01361072 -0.33824542  1.3603303 ]
  [-0.5960619   0.41967472 -0.42378655]]

 [[ 0.01361072 -0.33824542  1.3603303 ]
  [-1.1084248   0.24869247 -1.947749  ]]]
gradient w.r.t. embedding weights:
 [[0. 0. 0.]
 [2. 2. 2.]
 [1. 1. 1.]
 [0. 0. 0.]
 [1. 1. 1.]]


### 3.2 PyTorch
PyTorch 中，nn.Embedding 封装了上面所有逻辑，并加上了CUDA优化：

In [ ]:
import torch
import torch.nn as nn

# 1. 定义
embedding_layer = nn.Embedding(num_embeddings=10000, embedding_dim=256, padding_idx=0)
# 2. 输入必须是 LongTensor
input_indices = torch.tensor([[1, 2, 0], [4, 0, 5]], dtype=torch.long)
# 3. 前向传播
output = embedding_layer(input_indices)
print("Embedding output shape:", output.shape) # 应该是 (2, 3, 256)
# 4. 反向传播
output.sum().backward()
print("Gradient w.r.t. embedding weights shape:", embedding_layer.weight.grad.shape) # 应该是 (10000, 256)

## 面试官视角 (Interviewer's Eye)
Killer Question:
> “既然 One-Hot 编码会导致正交，使得向量之间没有语义相似度（点积为0），为什么在 Transformer 或 BERT 的输入层，我们还是先要把 Token 变成 One-Hot（或者说索引），再过 Embedding 层？为什么不直接输入某种预训练好的语义向量？”

S 级回答模板
> 1. 澄清概念： 我们实际上输入的是 Token ID（索引），数学上等价于 One-Hot 乘以 Embedding 矩阵。
> 2. 端到端学习 (End-to-End): 虽然 Word2Vec 等预训练向量有语义，但在特定任务（如法律文本分析 vs 社交媒体吐槽）中，词的语义是会漂移的。让模型通过 One-Hot 索引去从头学习（或微调） Embedding 矩阵 $W$，能让学到的向量空间最适配当前任务（Task-Specific）。
> 3. 基底的完备性： One-Hot 是一组完备的正交基。它提供了“区分所有词”的最纯粹信息。如果一开始就输入 Dense Vector，可能会引入先验偏差（Bias），限制了模型在高维空间中重新组织语义的能力。

## 下一步 (Next Step)
One-Hot 只是数据进入神经网络的第一步。为了处理高维稀疏数据（如文本），我们必须从 One-Hot 进化到 Distributed Representation (分布式表示)。建议下一步：A. Embedding Layer (嵌入层) - 手写查表操作，理解它如何把 One-Hot 压缩成 Dense Vector。B. Softmax & CrossEntropy - One-Hot 在损失函数中是如何作为 Label 存在的？（非常经典的梯度推导）